In [1]:
!pip install pandas pyarrow fastparquet

In [2]:
import pandas as pd

df = pd.read_parquet("hf://datasets/KAIST-IC-LAB721/SDCNL/data/train-00000-of-00001.parquet")

c:\Users\SabrinaLopez\anaconda3\envs\nlpproject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!pip install huggingface_hub

In [4]:
df

,title,text,label,label_text
0,Need help,Hi I don't really know how to phrase this situ...,0,depression
1,feeling so overwhelmed and hopeless,i have been so depressed these past couple wee...,1,suicidal
2,"Nothing matters anymore, getting worse",Hi..I don't know where else to go. I am devast...,0,depression
3,Who’s tired of hearing bullshit,"The shit like “it will get better, everyone is...",1,suicidal
4,I wish I was someone else.,I wish I was prettier. I wish I didn’t feel li...,0,depression
...,...,...,...,...
1890,think its over,i just don’t wanna live anymore so yeah,0,depression
1891,To all of those feeling isolated and suffering...,I’ve learned that life is fucking sad sometime...,0,depression
1892,I just really wish I had died the first time I...,That's all. Nothing has gotten better and I've...,1,suicidal
1893,I feel unimportant.,Not the first time I'm going through this of c...,0,depression


In [5]:
df['text'].head()

0    Hi I don't really know how to phrase this situ...
1    i have been so depressed these past couple wee...
2    Hi..I don't know where else to go. I am devast...
3    The shit like “it will get better, everyone is...
4    I wish I was prettier. I wish I didn’t feel li...
Name: text, dtype: object

In [6]:
for idx, str in enumerate(df['text']):
    # print(string)

    clean_str = list([val for val in str if val.isalnum() or val == ' ' or val == '.' or val == ',' or val == '?' or val == '!'])
    clean_str = ''.join(clean_str)
    low_clean_str = clean_str.lower()

    # print(low_clean_str)

    df = df.replace(df['text'][idx], low_clean_str)

In [7]:
df['text'][0]

'hi i dont really know how to phrase this situation but ill try. my life is at a really good point right now, im never really depressed over stuff and 99 percent of the time my mind is clear. im about to graduate high school and im really excited. however, people in my family and friend group have tons of issues. wether they sleep all day, hate themselves, or have no ambition to keep living on in this world, theyve got problems. i dont wanna sound like im gloating, but usually im the person that a lot of these people end up going to because usually im able to talk people through issues and help them in the long run. yeah sometimes their issues make me really sad and stuff because who doesnt feel sad when people are telling you they feel worthless.but today one of my best friends showed me that he was cutting and it really effected me. i talked to him about it and its mostly because of how painfully bored he is and he doesnt even know why hes doing it hes a pretty logical guy. he will g

In [8]:
!pip install transformers sentencepiece torch

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=2)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
!pip install tensorflow

In [11]:
example = tokenizer(['Hello World', 'hi how are you'], padding=True, truncation=True, return_tensors='pt')
example

{'input_ids': tensor([[    0, 35378,  6661,     2,     1,     1],
        [    0,  1274,  3642,   621,   398,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1]])}

In [12]:
!pip install torch

In [13]:
import torch

example['input_ids'] = example['input_ids'].type(torch.LongTensor) 
testing = model(**example)
testing

SequenceClassifierOutput(loss=None, logits=tensor([[-0.2215, -0.3197],
        [-0.2117, -0.3234]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [14]:
!pip install scikit-learn

In [15]:
from sklearn.model_selection import train_test_split

data_texts = df['text'].tolist()
data_labels = df['label'].tolist()
train_texts, temp_texts, train_labels, temp_labels = train_test_split(data_texts, data_labels, test_size=0.2, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)

print("Train set size:", len(train_texts))
print("Validation set size:", len(val_texts))
print("Test set size:", len(test_texts))

Train set size: 1516
Validation set size: 189
Test set size: 190


In [16]:
from torch.utils.data import Dataset

class SDCNLDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        inputs = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [17]:
train_dataset = SDCNLDataset(train_texts, train_labels, tokenizer)
val_dataset = SDCNLDataset(val_texts, val_labels, tokenizer)
test_dataset = SDCNLDataset(test_texts, test_labels, tokenizer)

In [18]:
!pip install accelerate>=0.26.0

In [21]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=lambda p: {
        "accuracy": accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1)),
        "precision": precision_score(p.label_ids, np.argmax(p.predictions, axis=1), average='binary'),
        "recall": recall_score(p.label_ids, np.argmax(p.predictions, axis=1), average='binary'),
        "f1": f1_score(p.label_ids, np.argmax(p.predictions, axis=1), average='binary'),
        "auroc": roc_auc_score(p.label_ids, p.predictions[:, 1])
    }
)

c:\Users\SabrinaLopez\anaconda3\envs\nlpproject\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\SabrinaLopez\AppData\Local\Temp\ipykernel_1512\3545177458.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
trainer.train()

























































































































                                                

                                          
                                            


 33%|███▎      | 95/285 [14:13<13:33,  4.28s/it]






{'eval_loss': 0.6389954686164856, 'eval_accuracy': 0.6931216931216931, 'eval_precision': 0.7280701754385965, 'eval_recall': 0.7545454545454545, 'eval_f1': 0.7410714285714286, 'eval_auroc': 0.728078250863061, 'eval_runtime': 9.5536, 'eval_samples_per_second': 19.783, 'eval_steps_per_second': 1.256, 'epoch': 1.0}

























































































































                                                
                                            


                                      

 33%|███▎      | 95/285 [20:40<13:33,  4.28s/it] 






{'eval_loss': 0.6752640008926392, 'eval_accuracy': 0.7248677248677249, 'eval_precision': 0.7685185185185185, 'eval_recall': 0.7545454545454545, 'eval_f1': 0.7614678899082569, 'eval_auroc': 0.7445339470655926, 'eval_runtime': 9.4503, 'eval_samples_per_second': 19.999, 'eval_steps_per_second': 1.27, 'epoch': 2.0}

























































































































                                                
                                            


                                      

 33%|███▎      | 95/285 [27:10<13:33,  4.28s/it] 






{'eval_loss': 0.7225422263145447, 'eval_accuracy': 0.6931216931216931, 'eval_precision': 0.7549019607843137, 'eval_recall': 0.7, 'eval_f1': 0.7264150943396226, 'eval_auroc': 0.7376294591484464, 'eval_runtime': 12.9497, 'eval_samples_per_second': 14.595, 'eval_steps_per_second': 0.927, 'epoch': 3.0}


                                                
                                            

100%|██████████| 285/285 [19:28<00:00,  4.10s/it]

{'train_runtime': 1168.3654, 'train_samples_per_second': 3.893, 'train_steps_per_second': 0.244, 'train_loss': 0.4883400900322094, 'epoch': 3.0}


TrainOutput(global_step=285, training_loss=0.4883400900322094, metrics={'train_runtime': 1168.3654, 'train_samples_per_second': 3.893, 'train_steps_per_second': 0.244, 'total_flos': 299157269944320.0, 'train_loss': 0.4883400900322094, 'epoch': 3.0})

In [23]:
evaluation_results = trainer.evaluate()
print("Evaluation Results:", evaluation_results)













100%|██████████| 12/12 [00:11<00:00,  1.06it/s]

Evaluation Results: {'eval_loss': 0.7225422263145447, 'eval_accuracy': 0.6931216931216931, 'eval_precision': 0.7549019607843137, 'eval_recall': 0.7, 'eval_f1': 0.7264150943396226, 'eval_auroc': 0.7376294591484464, 'eval_runtime': 12.3178, 'eval_samples_per_second': 15.344, 'eval_steps_per_second': 0.974, 'epoch': 3.0}


In [24]:
test_results = trainer.predict(test_dataset)
print("Test Results:", test_results.metrics)













100%|██████████| 12/12 [00:11<00:00,  1.05it/s]

Test Results: {'test_loss': 0.6350271701812744, 'test_accuracy': 0.7052631578947368, 'test_precision': 0.775, 'test_recall': 0.62, 'test_f1': 0.6888888888888889, 'test_auroc': 0.7912222222222222, 'test_runtime': 12.5336, 'test_samples_per_second': 15.159, 'test_steps_per_second': 0.957}
